In [1]:
!pip install wget
import wget
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

  Created wheel for wget: filename=wget-3.2-cp36-none-any.whl size=9682 sha256=f3a5db530234d5c859ebec7a07e761883218cdee9bd88fa08ae5296b79f42efc
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget


In [2]:
if not os.path.isdir("data/PetImages"):
    wget.download("https://oshi.at/oShhsA", bar=wget.bar_adaptive)

    import zipfile
    with zipfile.ZipFile("RfOr.zip", "r") as zip_ref:
        zip_ref.extractall("")


train_path = 'data/PetImages/train'
valid_path = 'data/PetImages/valid'

In [3]:
image_size = 224
batch_size = 10

train_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input) \
    .flow_from_directory(directory=train_path,
                         target_size=(image_size, image_size),
                         classes=['Cat', 'Dog'],
                         batch_size=batch_size)

valid_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input) \
    .flow_from_directory(directory=valid_path,
                         target_size=(image_size, image_size),
                         classes=['Cat', 'Dog'],
                         batch_size=batch_size)

Found 1000 images belonging to 2 classes.
Found 200 images belonging to 2 classes.


Wir verwenden nun das VGG16 Modell für Transfer Learning.
Unser Ziel ist es, ein neuronales Netz zu erstellen, welches Bilder von Katzen und Hunden unterscheiden kann. Dies geht mit einfachen CNNs nicht so gut, deshalb versuchen wir es mit Transfer Learning bei einem bereits trainierten neuronalen Netz, welches wir auf unsere Bedürfnisse spezialisieren.

In [4]:
vgg16_model = tf.keras.applications.vgg16.VGG16()
vgg16_model.summary()

553467904/553467096 [==============================] - 16s 0us/step
Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
___________________________________________________________

Wir erstellen nun ein eigenes Modell, das die gleichen Ebenen wie VGG16 hat außer der letzten.

In [5]:
model = Sequential()
for layer in vgg16_model.layers[:-1]:
    model.add(layer)

Da wir die Ebenen vor unserer Spezialisierung (dem letzten voll vernetzen neuronalen Netz) nicht erneut trainieren wollen, werden wir diese freezen, d. h. `trainable = False` setzen.

Statt dem Dense Layer mit 1000 Outputs verwenden wir ein Dense Layer mit 2 Outputs - Katze oder Hund.

In [6]:
for layer in model.layers:
    layer.trainable = False

In [7]:
model.add(Dense(units=2, activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)       2

Nun haben wir ein Modell, welches nur 8194 anpassbare Parameter hat statt 134 Millionen.
Wir verwenden wieder `Adam` als Optimizer und categorical crossentropy loss als loss funktion.

In [8]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [9]:
model.fit(x=train_batches, validation_data=valid_batches, epochs=3, verbose=2)

Epoch 1/3
100/100 - 550s - loss: 0.2233 - accuracy: 0.9240 - val_loss: 0.1995 - val_accuracy: 0.9250
Epoch 2/3
100/100 - 543s - loss: 0.0835 - accuracy: 0.9690 - val_loss: 0.1707 - val_accuracy: 0.9500
Epoch 3/3
100/100 - 544s - loss: 0.0513 - accuracy: 0.9850 - val_loss: 0.1777 - val_accuracy: 0.9450
